#### This file will demonstrate, and visualise the key functionalities of the Jax order book implementation. Further, it will measure the walltime speeds for these basic operations.

In [1]:
%load_ext autoreload
%autoreload 2
from gym_exchange.orderbook.orderbook import OrderBook
import gym_exchange.orderbook.test.genOrders as gen
import time
import random

ModuleNotFoundError: No module named 'gym_exchange'

In [36]:
def create_init_ob(eqsize=10):
    nlim=eqsize//3
    ob=OrderBook()
    buys=[]
    sells=[]
    id_init=900000
    time_init=34200.0
    for i in range(nlim):
        buys.append(gen.generate_new_buy(id_init,id_init,str(time_init)))
        sells.append(gen.generate_new_sell(id_init+1,id_init+1,str(time_init+1)))
        id_init+=2

    for order in buys+sells:
        ob.process_order(order,True,True)
    return ob
        


Measuring the time for the most basic operations: Adding and order and removing an order from a given side of the book.

Matching a single order against an identified order from the other side of the book:

Match against an entire side, continuing until either the full incoming order is matched, the book is empty, or the best price is no longer acceptable to the lim-order.

Matching is what takes the longest, and increases when the while loop needs to turn for longer. But even for a single iteration, it takes roughly 1.5 times the time than a simple add order.
Next we consider the higher-level message types and include the branching logic required to direct orders across different types and sides of orders.

In [37]:
random.seed(0)
n_books=1000
obs=[create_init_ob(100) for i in range(n_books)]

order=gen.generate_new_buy(8888,8888,'36000.124')
s=time.time()
for i,b in enumerate(obs):
    b.process_order(order,True,True)
en=time.time()
print((en-s)/len(obs))


7.060050964355468e-06


In [38]:
s=time.time()
for i,b in enumerate(obs):
    b.cancel_order('bid',8888,'37000.0')
en=time.time()
print((en-s)/len(obs))

3.4096240997314453e-06


In [39]:
trials=1000
rolling=0
#random.seed(0)
n_books=1000
obs=[create_init_ob(100) for i in range(n_books)]


for i in range(trials):
    order=gen.generate_cross_buy(7777,7777,'39000.125')
    #print(order)
    s=time.time()
    for i,b in enumerate(obs):
        b.process_order(order,True,False)
    en=time.time()
    rolling+=(en-s)
print((rolling)/len(obs)/trials)

7.573788404464722e-06


In [40]:
trials=1000
rolling=0
#random.seed(0)
n_books=1000
obs=[create_init_ob(100) for i in range(n_books)]

for i in range(trials):
    order=gen.generate_market_buy(6666,6666,'39500.125')
    #print(order)
    s=time.time()
    for i,b in enumerate(obs):
        b.process_order(order,True,False)
    en=time.time()
    rolling+=(en-s)
print((rolling)/len(obs)/trials)

1.899247646331787e-06
